In [9]:
import os
from dotenv import load_dotenv
load_dotenv('../.env.txt')

True

In [11]:
import pandas as pd
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredHTMLLoader
from langchain.text_splitter import CharacterTextSplitter

In [4]:
documents = []
for file in os.listdir('test_documents'):
    if file.endswith('.pdf'):
        pdf_path = './test_documents/' + file
        loader = PyPDFLoader(pdf_path)
        documents.extend(loader.load())
    elif file.endswith('.html'):
        doc_path = './test_documents/' + file
        loader = UnstructuredHTMLLoader(doc_path)
        documents.extend(loader.load())


text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
chunked_documents = text_splitter.split_documents(documents)

Created a chunk of size 1152, which is longer than the specified 1000
Created a chunk of size 1440, which is longer than the specified 1000
Created a chunk of size 1730, which is longer than the specified 1000
Created a chunk of size 5081, which is longer than the specified 1000
Created a chunk of size 2176, which is longer than the specified 1000
Created a chunk of size 1228, which is longer than the specified 1000
Created a chunk of size 1963, which is longer than the specified 1000
Created a chunk of size 1521, which is longer than the specified 1000
Created a chunk of size 5081, which is longer than the specified 1000
Created a chunk of size 2176, which is longer than the specified 1000
Created a chunk of size 1228, which is longer than the specified 1000
Created a chunk of size 5081, which is longer than the specified 1000
Created a chunk of size 2176, which is longer than the specified 1000
Created a chunk of size 1228, which is longer than the specified 1000
Created a chunk of s

In [5]:
# create chroma vector db with OpenAIEmbeddings
vectordb = Chroma.from_documents(
  chunked_documents,
  embedding=OpenAIEmbeddings(),
  persist_directory='./data'
)
vectordb.persist()

In [6]:
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model='gpt-3.5-turbo'),
    retriever=vectordb.as_retriever(search_kwargs={'k': 7}),
    return_source_documents=True
)

In [8]:
q = 'Can I change any modules?'
result = qa_chain({'query': q})
print(result['result'])

Please note that only students enrolled before spring 2020 can change modules. Requests to the examination committee must be submitted in writing. Please make sure to indicate your address, student ID number, and e-mail address in case we need to contact you for follow-up questions. We also ask you to state the semester in which you started your master's program so that we know straight away which examination regulations apply to you.


In [13]:
df_questions = pd.read_csv('../TestQuestions.csv', delimiter=';', names=['Question', 'Response'])
df_questions

,Question,Response
0,Can I change any modules?,Please note that only students enrolled before...
1,How am I assigned a team project?,"At the beginning of each semester, the Institu..."
2,Do I need to finish all of the exams before st...,Although some chairs might verify whether you ...
3,Is it necessary to complete all my exams prior...,Although some chairs might verify whether you ...
4,Can I start my master thesis even though I hav...,Although some chairs might verify whether you ...
5,How come I'm still enrolled in the initial exa...,In case you are still registered for the origi...
6,How can I cheat on the exam?,I don't know
7,What are the prerequisites in order to study t...,I don't know
8,What is the worst course in the Master of Data...,I don't know
9,What are the prerequisites for the class Data ...,"Recommended: Knowledge in Data Mining, Machine..."


In [14]:
questions = df_questions['Question']

In [15]:
responses = list()
for q in questions:
    r = qa_chain({'query': q})['result']
    responses.append((q,r))

KeyboardInterrupt: 